
# 06. 최종 앙상블 및 제출 (Final Submission)

본 노트북은 앞서 학습된 **CatBoost, XGBoost, FiBiNet** 3가지 단일 모델의 결과를 결합하여 최종 예측 성능을 극대화합니다.
각 모델의 특성과 검증 점수를 반영한 **가중 앙상블(Weighted Ensemble)** 전략을 사용합니다.

### 🏆 앙상블 전략
1. **모델 가중치 배분**: 
    - **CatBoost**: 40% (범주형 데이터 처리 및 안정성 우수)
    - **XGBoost**: 35% (강력한 패턴 인식 및 과적합 제어)
    - **FiBiNet**: 25% (비선형 상호작용 및 다양성 확보)
2. **Soft Voting**: 각 모델이 예측한 확률값(Probability)의 가중 평균 계산
3. **Submission**: 최종 예측 결과를 `final_submission.csv`로 저장


In [ ]:

import pandas as pd
import os

# 파일명 정의
files = {
    'catboost': '03_CatBoost_SoftVoting.csv',
    'xgboost': '04_XGBoost_SoftVoting.csv',
    'fibinet': '05_FiBiNet_SoftVoting.csv'
}

# 가중치 정의
weights = {
    'catboost': 0.40,
    'xgboost': 0.35,
    'fibinet': 0.25
}

print("📋 설정된 가중치:", weights)


In [ ]:

# 1. 파일 로드 및 병합
dfs = {}
for model_name, filename in files.items():
    if os.path.exists(filename):
        print(f"✅ Loaded {model_name}: {filename}")
        dfs[model_name] = pd.read_csv(filename)
    else:
        print(f"⚠️ Missing {model_name}: {filename} (Skipping...)")

if not dfs:
    print("❌ 병합할 파일이 없습니다.")
else:
    # 첫 번째 데이터프레임을 기준으로 병합 시작
    base_model = list(dfs.keys())[0]
    merged = dfs[base_model].rename(columns={'clicked': f'clicked_{base_model}'})
    
    for model_name in list(dfs.keys())[1:]:
        df_temp = dfs[model_name].rename(columns={'clicked': f'clicked_{model_name}'})
        merged = merged.merge(df_temp, on='ID')
        
    # 2. 가중 평균 계산
    # 실제 로드된 모델의 가중치만 재조정 (합이 1이 되도록)
    idx_sanity_check = list(dfs.keys()) # 로드된 모델 리스트
    current_weights = {k: weights[k] for k in idx_sanity_check}
    total_w = sum(current_weights.values())
    
    print(f"\n⚖️ 원본 가중치 합: {total_w:.2f} -> 정규화 적용")
    
    merged['clicked'] = 0
    for model_name in idx_sanity_check:
        norm_w = current_weights[model_name] / total_w
        merged['clicked'] += merged[f'clicked_{model_name}'] * norm_w
        print(f"   + {model_name}: {norm_w:.4f}")
        
    # 3. 저장
    output_path = '06_Final_Submission.csv'
    merged[['ID', 'clicked']].to_csv(output_path, index=False)
    print(f"\n🎉 최종 앙상블 완료! 파일 저장됨: {output_path}")
    print(merged[['ID', 'clicked']].head())
